In [ ]:
# Install GDAL and deepdespeckling
!apt-get install -y gdal-bin python-gdal python3-gdal
!pip install deepdespeckling


In [ ]:
from deepdespeckling.merlin.merlin_denoiser import MerlinDenoiser
import numpy as np
import matplotlib.pyplot as plt
import cv2  # To read PNG images in grayscale format

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
import numpy as np
import os
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim, mean_squared_error as mse
import matplotlib.pyplot as plt

# Directories
input_folder = '/content/drive/MyDrive/test/sar_tif'       # Folder with original SAR images
output_folder = '/content/drive/MyDrive/Merlin_output/'     # Folder to save denoised images
expected_folder = '/content/drive/MyDrive/test/gray_tif' # Folder with expected denoised images

# Ensure output directory exists
os.makedirs(output_folder, exist_ok=True)

# Initialize lists to hold PSNR, SSIM, and MSE values for all images
psnr_values = []
ssim_values = []
mse_values = []

# Set parameters for MERLIN
patch_size = 32
stride_size = 8
symmetrize = False
model_type = "Sentinel-TOPS"

# Initialize MERLIN denoiser
merlin_denoiser = MerlinDenoiser(model_name=model_type, symetrise=symmetrize)

# Specify image for display after denoising
display_image_name = "/content/drive/MyDrive/907.tif"  # Replace with the specific filename to display

# Process each image in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".tif") or filename.endswith(".png"):  # Adjust formats as needed
        # Load input image
        input_image_path = os.path.join(input_folder, filename)
        image = cv2.imread(input_image_path, cv2.IMREAD_GRAYSCALE)

        if image is None:
            print(f"Error loading {filename}")
            continue

        # Convert to float32 and stack channels for MERLIN
        image = image.astype(np.float32)
        image = np.stack((image, image), axis=-1)

        # Perform MERLIN denoising
        denoised_image_merlin = merlin_denoiser.denoise_image(image, patch_size=patch_size, stride_size=stride_size)
        denoised_image = denoised_image_merlin["denoised"]["full"]

        # Save denoised image
        output_image_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_image_path, denoised_image.astype(np.uint8))

        # Load expected image for comparison
        expected_image_path = os.path.join(expected_folder, filename)
        expected_image = cv2.imread(expected_image_path, cv2.IMREAD_GRAYSCALE)

        if expected_image is not None:
            # Calculate PSNR, SSIM, and MSE for this image pair
            psnr_value = psnr(expected_image, denoised_image)
            ssim_value = ssim(expected_image, denoised_image, data_range=denoised_image.max() - denoised_image.min())
            mse_value = mse(expected_image, denoised_image)

            # Append to lists
            psnr_values.append(psnr_value)
            ssim_values.append(ssim_value)
            mse_values.append(mse_value)
        else:
            print(f"Expected image not found for {filename}")

        # Display the original, denoised, and expected image for the specified image
        if filename == display_image_name:
            plt.figure(figsize=(15, 5))

            # Original Image
            plt.subplot(1, 3, 1)
            plt.title("Original Image")
            plt.imshow(cv2.imread(input_image_path, cv2.IMREAD_GRAYSCALE), cmap='gray')
            plt.axis("off")

            # Denoised Image
            plt.subplot(1, 3, 2)
            plt.title("MERLIN Denoised Image")
            plt.imshow(denoised_image, cmap='gray')
            plt.axis("off")

            # Expected Image
            if expected_image is not None:
                plt.subplot(1, 3, 3)
                plt.title("Expected Image")
                plt.imshow(expected_image, cmap='gray')
                plt.axis("off")

            plt.show()

# Calculate average PSNR, SSIM, and MSE across all images
average_psnr = np.mean(psnr_values)
average_ssim = np.mean(ssim_values)
average_mse = np.mean(mse_values)

# Display results
print(f"Average PSNR: {average_psnr:.2f}")
print(f"Average SSIM: {average_ssim:.4f}")
print(f"Average MSE: {average_mse:.2f}")
